<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100"/>

<div>
    <font color=#690027 markdown="1">   
        <h1>CLASSIFICATIE STOMATA OP BEZONDE EN BESCHADUWDE BLADEREN MET SCIKIT-LEARN</h1> 
    </font>
</div>

<div class="alert alert-box alert-success">
In deze notebook gebruik je een functie en methodes van de Python-module scikit-learn om bezonde en beschaduwde bladeren te classificeren. 
</div>

Krappa of crabwood is een snelgroeiende boomsoort die veelvuldig voorkomt in het Amazonegebied. Volwassen exemplaren kunnen een diameter hebben van meer dan een meter en kunnen meer dan 40 meter hoog zijn. <br>Het hout van hoge kwaliteit wordt gebruikt voor het maken van meubelen, vloeren, masten ... Uit de schors wordt een koortswerend middel gehaald. Uit de zaden produceert men een olie voor medicinale toepassingen, waaronder de behandeling van huidziekten en tetanos, en als afweermiddel voor insecten. 

<table><tr>
<td> <img src="images/andirobaamazonica.jpg" alt="Drawing" width="200"/></td>
<td> <img src="images/crabwoodtree.jpg" alt="Drawing" width="236"/> </td>
</tr></table>

<center>
Foto's: Mauroguanandi [Public domain] [2] en P. S. Sena [CC BY-SA 4.0] [3].
</center>

Omdat sommige klimaatmodellen een stijging van de temperatuur en een vermindering in regenval voorspellen in de komende decennia, is het belangrijk om te weten hoe deze bomen zich aanpassen aan veranderende omstandigheden. <br>
Wetenschappers Camargo en Marenco deden onderzoek in het Amazonewoud [1].<br>
Naast de invloed van seizoensgebonden regenval bekeken ze ook stomatale kenmerken van bladeren onder bezonde en onder beschaduwde condities.<br> Hiervoor werden een aantal planten, opgekweekt in de schaduw, verplaatst naar vol zonlicht gedurende 60 dagen. Een andere groep planten werd in de schaduw gehouden. <br>De kenmerken van de stomata werden opgemeten op afdrukken van de bladeren gemaakt met transparante nagellak. 

### Nodige modules importeren

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression       # voor classificatie met Logistic Regression Classifier

from matplotlib import animation                          # om animatie te maken
from IPython.display import HTML                          # om animatie te laten zien in notebook

<div>
    <font color=#690027 markdown="1">   
        <h2>1. Inlezen van de data</h2> 
    </font>
</div>

Lees met de module `pandas` de dataset in.

In [ ]:
stomata = pd.read_csv("data/schaduwzon.csv", header="infer")  # in te lezen tabel heeft hoofding

In [ ]:
stomata

<div>
    <font color=#690027 markdown="1">   
        <h2>2. De data inlezen en voorbereiden</h2> 
    </font>
</div>

Net zoals in de twee andere notebooks zet je de dichtheid van de stomata uit in functie van de lengte van de stomata, en zal je werken met gestandaardiseerde data. 

In [ ]:
# data
x1 = stomata["stomatale lengte"]          # kenmerk: lengte
x2 = stomata["stomatale dichtheid"]       # kenmerk: dichtheid
x1 = np.array(x1)          # kenmerk: lengte
x2 = np.array(x2)          # kenmerk: dichtheid

# standaardiseren
x1 = (x1 - np.mean(x1)) / np.std(x1)
x2 = (x2 - np.mean(x2)) / np.std(x2)

y = stomata["milieu"]            # labels: tweede kolom van oorspronkelijke tabel
y = np.array(y)
y = np.where(y == "zon", 1, 0)   # labels numeriek maken, zon:1, schaduw:0

In [ ]:
X = np.stack((x1, x2), axis = 1)    # juiste formaat, namelijk NumPy array

In [ ]:
# dataset met input X(x1, x2) en output y
print(X)
print(X.shape, type(X))
print(y)

<div>
    <font color=#690027 markdown="1">   
        <h2>3. Classificatie</h2> 
    </font>
</div>

<div>
    <font color=#690027 markdown="1">   
        <h3>3.1 Classificatie uitvoeren met scikit-learn</h3> 
    </font>
</div>

In [ ]:
# classificatie met scikit-learn Logistic Regression Classifier
clf = LogisticRegression(multi_class="multinomial")
clf.fit(X,y)                                # classificatie toepassen op data

<div>
    <font color=#690027 markdown="1">   
        <h3>3.2 Classificatie testen</h3> 
    </font>
</div>

In [ ]:
# test a.d.h.v. trainingset (beter zou zijn via testset)
datapunt1 = [X[5]]
clf.predict(datapunt1) 

In [ ]:
# representatie bekijken
print(X[5])
print(datapunt1)

In [ ]:
# test a.d.h.v. trainingset (beter zou zijn via testset)
datapunt2 = [X[30]]
clf.predict(datapunt2) 

<div>
    <font color=#690027 markdown="1">   
        <h3>3.3 Classificatie visueel voorstellen (eerste manier)</h3> 
    </font>
</div>

In [ ]:
# bereik assen
x_min, x_max = x1.min() - 0.5, x1.max() + 0.5
y_min, y_max = x2.min() - 0.5, x2.max() + 0.5

h = 0.01      # stap, resolutie 
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))     # rooster maken met punten (xx, yy)
print(xx)
print(xx.shape)
print(yy.shape)

# np.c_[xx.ravel(), yy.ravel() is alle mogelijke coördinaten gevormd met xx'n en yy's
roosterpunten = np.c_[xx.ravel(), yy.ravel()]
print(roosterpunten)

In [ ]:
# van al deze punten (xx, yy) klasse voorspellen
z = clf.predict(roosterpunten)     

print(z)

In [ ]:
# voorspellingen z staan in rijmatrix, elke voorspelling moet overeenkomen met punt in rooster
z = z.reshape(xx.shape)   

print(z)
print(z.shape)

In [ ]:
# resultaat plotten

plt.figure(figsize=(10, 8))

# plot datapunten
plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn volgende 25

# plot scheidingslijn
# voorspelling z heeft voor elk punt in rooster waarde 0 of 1, rooster wordt zo in twee delen verdeeld
# grens tussen twee gebieden wordt getekend in zwart 
plt.contour(xx, yy, z, colors="black")            

plt.title("Carapa (gestandaardiseerd)")
plt.xlabel("stomatale lengte")
plt.ylabel("stomatale densiteit")
plt.legend(loc="lower left")

plt.xlim(xx.min()-0.5, xx.max()+0.5)
plt.ylim(yy.min()-0.5, yy.max()+0.5)

plt.show()

<div>
    <font color=#690027 markdown="1">   
        <h3>3.4 Classificatie visueel voorstellen (tweede manier)</h3> 
    </font>
</div>

In [ ]:
# andere voorstelling resultaat plotten

plt.figure(figsize=(10, 8))

# plot datapunten
plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn volgende 25

# rooster, resolutie
xx1 = np.arange(x1.min()-1, x1.max()+1, 0.1)
xx2 = np.arange(x2.min()-1, x2.max()+1, 0.1) 

for xcoord in xx1:
    for ycoord in xx2:
        voorspelling = clf.predict([[xcoord, ycoord]])  
        if voorspelling == 1:
            plt.plot(xcoord, ycoord, marker=".", color="lightblue")
        else:
            plt.plot(xcoord, ycoord, marker=".", color="orange")

plt.title("Carapa (gestandaardiseerd)")
plt.xlabel("stomatale lengte")
plt.ylabel("stomatale densiteit")
plt.legend(loc="lower left")

plt.xlim(xx.min()-0.5, xx.max()+0.5)
plt.ylim(yy.min()-0.5, yy.max()+0.5)

plt.show()

<div>
    <h2>Referentielijst</h2> 
</div>

[1] Camargo, Miguel Angelo Branco, & Marenco, Ricardo Antonio. (2012). Growth, leaf and stomatal traits of crabwood (*Carapa guianensis Aubl.*)<br> &nbsp; &nbsp; &nbsp; &nbsp; in central Amazonia. Revista Árvore, 36(1), 07-16. https://dx.doi.org/10.1590/S0100-67622012000100002 en via e-mail.<br>
[2] Mauroguanandi [Public domain]. https://commons.wikimedia.org/wiki/File:Andirobaamazonica.jpg. <br> &nbsp; &nbsp; &nbsp; &nbsp; Geraadpleegd op 13 augustus 2019 via Wikimedia Commons. <br>
[3] Sena, P. S. https://commons.wikimedia.org/wiki/File:Crabwood_tree.JPG. [CC BY-SA 4.0] Geraadpleegd op 13 augustus 2019 via Wikimedia Commons.

<div>
    <h2>Met steun van</h2> 
</div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>.,